LEIA-ME
Fala Professor, tudo bem ?
Então aprovetei o tempo vaga e dei uma estuda em novas funções e novas funcionalidades no meio do código misturei algumas informações aleátorias para colocar mais funcionalidade a lista de dados extraída do Kaggle, afim de deixar mais completo o trabalho. 
Importante citar que no meio do notebook é necessário selecionar em forma de check box a itenção de filtro, para isso dar continuidade a execução. 
A base de dados é a que segue no link : https://www.kaggle.com/datasets/middlelight/goodreadsbookswithgenres?resource=download
 é importante dizer que não a inserção dos generos feitos por este usuario não condiz fielmente ao que é de fato, mas é interessante pegar uma base assim um pouco mais confusa para testar modelos de recomendação e qual se adequa melhor. 
 O código está na grande maioria comentado linha a linha quando é um algoritimo mais complexo e quando é uma coisa mais simples fiz um comentário externo. 
 Obs.: Foram feitas 3 recomendações: 
 1- recomendação do perfil do usuario
 2- recomendação do perfil menos os que ele já assitiu
 3 - recomendação de acordo com os filmes já assistindo pegando pelo titulo 
 Agradeço pelos conhecimentos passados! 
 Se ficar alguma dúvida estou a disposição. 

Importando as bibliotecas necessárias

In [48]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from ipywidgets import widgets
from functools import reduce
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Fazendo a leitura do arquivo CSV forneceido no kaggle, segue link : https://www.kaggle.com/datasets/middlelight/goodreadsbookswithgenres?resource=download

In [49]:
base = pd.read_csv("Goodreads_books_with_genres.csv")


In [50]:
base.head()

,Book Id,Title,Author,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,genres
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Adve..."


verfiicando as colunas do arquivo e entendo mais sobre

In [51]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             11127 non-null  int64  
 1   Title               11127 non-null  object 
 2   Author              11127 non-null  object 
 3   average_rating      11127 non-null  float64
 4   isbn                11127 non-null  object 
 5   isbn13              11127 non-null  int64  
 6   language_code       11127 non-null  object 
 7   num_pages           11127 non-null  int64  
 8   ratings_count       11127 non-null  int64  
 9   text_reviews_count  11127 non-null  int64  
 10  publication_date    11127 non-null  object 
 11  publisher           11127 non-null  object 
 12  genres              11030 non-null  object 
dtypes: float64(1), int64(5), object(7)
memory usage: 1.1+ MB


renomeando as colunas para tornar a visualização mais pratica

In [52]:
# renomeia as colunas
base = base.rename(columns={
    "Id": "COD_LIVRO",
    "Title": "TITULO",
    "Author": "Autor",
    "average_rating": "NOTA_MEDIA",
    "language_code": "IDIOMA",
    "num_pages": "NUM_TOTAL_PAG",
    "ratings_count": "QTD_TOTAL_AVALICAO",
    "text_reviews_count": "QTD_REVIEWS",
    "publication_date": "DATA_PUBLICACAO",
    "publisher": "EDITOR",
    "genres": "GENEROS"
})

# salva o dataset com as colunas renomeadas
base.to_csv("filmes.csv", index=False)

In [53]:
base1 = pd.read_csv('filmes.csv')

In [54]:
base1.head()

,Book Id,TITULO,Autor,NOTA_MEDIA,isbn,isbn13,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Adve..."


retirando colunas que serão úteis

In [55]:
base1 = base1.drop(columns=["isbn", "isbn13"])

In [56]:
base1

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Chil..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,11/1/2003,Scholastic,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,"Fantasy;Fiction;Young Adult;Fantasy,Magic;Chil..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,"Fantasy;Young Adult;Fiction;Fantasy,Magic;Adve..."
...,...,...,...,...,...,...,...,...,...,...,...
11122,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,eng,512,156,20,12/21/2004,Da Capo Press,"Fiction;Writing,Essays;Literature,American;The..."
11123,45633,You Bright and Risen Angels,William T. Vollmann,4.08,eng,635,783,56,12/1/1988,Penguin Books,Fiction;Science Fiction;Literature;Novels;Lite...
11124,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,eng,415,820,95,8/1/1993,Penguin Books,"Historical,Historical Fiction;Fiction;Novels;F..."
11125,45639,Poor People,William T. Vollmann,3.72,eng,434,769,139,2/27/2007,Ecco,"Nonfiction;Sociology;Social Issues,Poverty;His..."


In [57]:
base1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             11127 non-null  int64  
 1   TITULO              11127 non-null  object 
 2   Autor               11127 non-null  object 
 3   NOTA_MEDIA          11127 non-null  float64
 4   IDIOMA              11127 non-null  object 
 5   NUM_TOTAL_PAG       11127 non-null  int64  
 6   QTD_TOTAL_AVALICAO  11127 non-null  int64  
 7   QTD_REVIEWS         11127 non-null  int64  
 8   DATA_PUBLICACAO     11127 non-null  object 
 9   EDITOR              11127 non-null  object 
 10  GENEROS             11030 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 956.4+ KB


tirando as linhas que contém branco ou nulo

In [58]:
base1 = base1.dropna()


In [59]:
base1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11030 entries, 0 to 11126
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             11030 non-null  int64  
 1   TITULO              11030 non-null  object 
 2   Autor               11030 non-null  object 
 3   NOTA_MEDIA          11030 non-null  float64
 4   IDIOMA              11030 non-null  object 
 5   NUM_TOTAL_PAG       11030 non-null  int64  
 6   QTD_TOTAL_AVALICAO  11030 non-null  int64  
 7   QTD_REVIEWS         11030 non-null  int64  
 8   DATA_PUBLICACAO     11030 non-null  object 
 9   EDITOR              11030 non-null  object 
 10  GENEROS             11030 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 1.0+ MB


editando a coluna de generos para ficar mais fácil a pesquisa

In [60]:
base1["GENEROS"] = base1["GENEROS"].str.replace(";", "|")
base1["GENEROS"] = base1["GENEROS"].str.replace(",", "|")

<ipython-input-60-ed3a24a08710>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base1["GENEROS"] = base1["GENEROS"].str.replace(";", "|")
<ipython-input-60-ed3a24a08710>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base1["GENEROS"] = base1["GENEROS"].str.replace(",", "|")


In [61]:
base1.head()

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,11/1/2003,Scholastic,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,Fantasy|Young Adult|Fiction|Fantasy|Magic|Adve...


agora vamos definir o peso de cada avaliação para criterio de escolha do filme
como não temos voto a voto apenas o nome do filme e nota média das avaliações, vamos cortar os filmes que não estão inclusos no 3 quartil para baixo das avaliações, ou seja, que tem menos de 25 % da média das avaliações. 
para isso o codigo abaixo:


In [62]:
primeiro_quartil = base["QTD_TOTAL_AVALICAO"].quantile(0.25)

In [63]:
base1 = base1.loc[base["QTD_TOTAL_AVALICAO"] >= primeiro_quartil]

In [64]:
base1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8352 entries, 0 to 11126
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             8352 non-null   int64  
 1   TITULO              8352 non-null   object 
 2   Autor               8352 non-null   object 
 3   NOTA_MEDIA          8352 non-null   float64
 4   IDIOMA              8352 non-null   object 
 5   NUM_TOTAL_PAG       8352 non-null   int64  
 6   QTD_TOTAL_AVALICAO  8352 non-null   int64  
 7   QTD_REVIEWS         8352 non-null   int64  
 8   DATA_PUBLICACAO     8352 non-null   object 
 9   EDITOR              8352 non-null   object 
 10  GENEROS             8352 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 783.0+ KB


In [65]:
base1.head()

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,eng,352,6333,244,11/1/2003,Scholastic,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,Fantasy|Young Adult|Fiction|Fantasy|Magic|Adve...


In [66]:
base.limpa = base1

<ipython-input-66-850e01d3a537>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  base.limpa = base1


pronto, agora retirado os filmes que na teoria são pouco avaliados, vamos realizar a primeira maneira de recomendação que é a simplificada, utilizando apenas o genero, ordenando a recomendação por maior NOTA_MEDIA depois por QTD_TOTAL_AVALIACAO.

criando a possibilidade do usuario selecionar o perfil dele, criando assim uma adequação as indicações

In [67]:
texto_completo = '|'.join(base1['GENEROS'].tolist())

# Divide a string em palavras separadas pelo caractere '|'
palavras = texto_completo.split('|')

# Conta a quantidade de palavras únicas
palavras_unicas = set(palavras)
#Crio um dicionário para calcular a frequencia de repetição das palavras
frequencia_palavras = {}
for palavra in palavras:
    if palavra in frequencia_palavras:
        frequencia_palavras[palavra] += 1
    else:
        frequencia_palavras[palavra] = 1
#Aqui encerro a contagem de cada palavra e identifico quantas palavras unicas existem
total_palavras = len(palavras)
frequencia_relativa = {}
#aqui começo a fazer a conta para descobrir as palavras que tem mais do que 75% da repetição. 
for palavra, frequencia in frequencia_palavras.items():
    frequencia_relativa[palavra] = frequencia / total_palavras
frequencia_relativa_media = sum(frequencia_relativa.values()) / len(frequencia_relativa)
palavras_relevantes = [palavra for palavra, frequencia in frequencia_relativa.items() if frequencia > 0.75 * frequencia_relativa_media]
print("As seguintes palavras aparecem mais do que 75% da frequência relativa média:")
print(palavras_relevantes)

As seguintes palavras aparecem mais do que 75% da frequência relativa média:
['Fantasy', 'Young Adult', 'Fiction', 'Magic', 'Childrens', 'Adventure', 'Audiobook', 'Middle Grade', 'Classics', 'Science Fiction Fantasy', 'Supernatural', 'Mystery', 'Paranormal', 'Novels', 'Science Fiction', 'Humor', 'Comedy', 'European Literature', 'British Literature', 'Nonfiction', 'Science', 'History', 'Historical', 'Philosophy', 'Unfinished', 'Travel', 'Cultural', 'Africa', 'Autobiography', 'Memoir', 'Biography', 'Language', 'Writing', 'Humanities', 'Reference', 'Books About Books', 'Essays', 'Literature', 'American', 'The United States Of America', 'Epic Fantasy', 'High Fantasy', 'Art', 'Media Tie In', 'Culture', 'Adult', 'Computer Science', 'Academic', 'School', 'Realistic Fiction', 'Read For School', 'Biography Memoir', 'Romance', 'Anthologies', 'Adult Fiction', 'Paranormal Romance', 'Urban Fantasy', 'Mythology', 'Fairy Tales', 'Juvenile', 'Coming Of Age', 'Historical Fiction', 'Medieval', 'Literary

Apesar de ter filtrado mais do que 75% ainda sobram muitas palavras, mas como já é um numero ok, manterei, mas para diminuir basta aumentar a % no codigo acima

In [68]:
len(palavras_relevantes)

161

In [72]:
opcoes = palavras_relevantes
selecao = widgets.SelectMultiple(options=opcoes, description='Selecione uma opção:(Segure control para selecionar mais do que um)')
display(selecao)

SelectMultiple(description='Selecione uma opção:(Segure control para selecionar mais do que um)', options=('Fa…

transformando o perfil de usuario em uma lista para indicação dos melhores filmes

In [85]:
PERFIL_USUARIO = selecao.value
print("Você selecionou os generos:", PERFIL_USUARIO)

Você selecionou os generos: ('Fantasy', 'Fiction', 'Magic')


executando script para filtrar somente filmes para o usuario de acordo com o perfil dele

In [86]:
mascaras = [base1['GENEROS'].apply(lambda x: pref in x) for pref in PERFIL_USUARIO] #aqui aplico a função lambda com for para buscar os generos existentes e relaciona-los ao perfil 
mask = reduce(lambda x, y: x & y, mascaras)#é utilizado a função reduce para aplicar o "AND" ao invez de "OR" afim de ter um resultado mais exato do que o usuário tem relação
recomendados = base1[mask]
recomendados = recomendados.sort_values(['NOTA_MEDIA', 'QTD_TOTAL_AVALICAO'], ascending=False)#aqui eu listo primeiro por nota media e como criterio de desempate a qtd de avaliacao


exibindo somente os 10 primeiros

In [87]:
recomendados.head(10)

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS,LIDOS
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,eng,2690,41428,164,9/13/2004,Scholastic,Fantasy|Young Adult|Fiction|Fantasy|Magic|Adve...,Sim
6,10,Harry Potter Collection (Harry Potter #1-6),J.K. Rowling,4.73,eng,3342,28242,808,9/12/2005,Scholastic,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...,Não
21,30,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,J.R.R. Tolkien,4.59,eng,1728,101233,1550,9/25/2012,Ballantine Books,Fantasy|Fiction|Classics|Adventure|Science Fic...,Sim
4273,15336,The Lord of the Rings / The Hobbit,J.R.R. Tolkien,4.59,eng,1600,141,5,10/7/2002,Collins Modern Classics,Fantasy|Fiction|Classics|Adventure|Science Fic...,Não
4970,17961,Collected Fictions,Jorge Luis Borges/Andrew Hurley,4.58,eng,565,18874,791,9/30/1999,Penguin Classics Deluxe Edition,Fiction|Short Stories|Classics|Literature|Fant...,Não
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...,Não
4412,15872,Harry Potter y el misterio del príncipe (Harry...,J.K. Rowling/Gemma Rovira Ortega,4.57,spa,602,5104,386,2/28/2006,Salamandra,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...,Não
615,2005,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling,4.57,eng,768,1213,78,6/23/2006,Bloomsbury Publishing,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...,Não
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,Fantasy|Fiction|Young Adult|Fantasy|Magic|Chil...,Sim
10679,43509,Harry Potter and the Goblet of Fire (Harry Pot...,J.K. Rowling,4.56,eng,636,18754,906,7/8/2000,Bloomsbury,Fantasy|Young Adult|Fiction|Fantasy|Magic|Chil...,Não


In [88]:
recomendados.info()
#veja que apesar de ter uma quantidade maior de filmes que atendem as expectativas do usuario, foram lista somente os 10 primeiros.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 4 to 9966
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             486 non-null    int64  
 1   TITULO              486 non-null    object 
 2   Autor               486 non-null    object 
 3   NOTA_MEDIA          486 non-null    float64
 4   IDIOMA              486 non-null    object 
 5   NUM_TOTAL_PAG       486 non-null    int64  
 6   QTD_TOTAL_AVALICAO  486 non-null    int64  
 7   QTD_REVIEWS         486 non-null    int64  
 8   DATA_PUBLICACAO     486 non-null    object 
 9   EDITOR              486 non-null    object 
 10  GENEROS             486 non-null    object 
 11  LIDOS               486 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 49.4+ KB


Agora para começar a misturar, vamos criar uma lista dos livros que o usuario já leu, para evitar que ele assista os mesmo filmes, para não dar sempre o mesmo resultado vamos fazer um algoritimo aleatorio, que gere 100 numeros entre 1 e 11127 que é a quantidade de filmes originalmente listados no dataset.

In [89]:
HISTORICO = []
for i in range(11120):
    HISTORICO.append(random.randint(1, 11127))

In [90]:
HISTORICO

[353,
 3062,
 6019,
 4879,
 8612,
 8110,
 5664,
 7867,
 3831,
 8259,
 7040,
 1081,
 2251,
 3259,
 630,
 3622,
 4880,
 707,
 8976,
 4346,
 6930,
 7844,
 6472,
 3067,
 3336,
 1224,
 445,
 2495,
 8101,
 571,
 4819,
 8109,
 1574,
 5624,
 4298,
 6132,
 2992,
 8613,
 1839,
 1005,
 4941,
 4562,
 3596,
 1523,
 4689,
 3230,
 800,
 8840,
 3226,
 5362,
 3111,
 9179,
 10670,
 998,
 7865,
 6353,
 1918,
 1296,
 9046,
 9701,
 9606,
 8974,
 6017,
 3322,
 3493,
 5941,
 835,
 4815,
 11046,
 6708,
 8306,
 7121,
 4399,
 4309,
 8159,
 10297,
 8577,
 9579,
 4775,
 3958,
 2415,
 1784,
 5392,
 922,
 8806,
 1980,
 10573,
 8548,
 5994,
 989,
 2343,
 10502,
 6056,
 5344,
 10702,
 10190,
 5070,
 7870,
 9601,
 2754,
 5600,
 3922,
 6999,
 8968,
 3665,
 5883,
 1121,
 2533,
 8330,
 2996,
 4356,
 6610,
 9299,
 1829,
 7578,
 1315,
 10804,
 4188,
 4509,
 384,
 2422,
 787,
 9763,
 8122,
 6549,
 8490,
 2491,
 5789,
 6625,
 1949,
 687,
 2092,
 9663,
 1978,
 2420,
 6780,
 6692,
 1283,
 1029,
 3111,
 1601,
 2847,
 8214,
 256

agora vamos tratar a recomendação dos livros que ele tem relação com o perfil menos os que ele já leu

In [91]:
base1['LIDOS'] = np.where(base1['Book Id'].isin(HISTORICO), 'Sim', 'Não')

In [92]:
base1['LIDOS']

0        Sim
1        Sim
2        Não
3        Não
4        Não
        ... 
11122    Não
11123    Não
11124    Não
11125    Não
11126    Não
Name: LIDOS, Length: 8352, dtype: object

In [99]:
base1['LIDOS'].value_counts()

Não    6895
Sim    1457
Name: LIDOS, dtype: int64

In [97]:
base1['Book Id']= base1['Book Id'].astype(int)
recomendados['Book Id']= recomendados['Book Id'].astype(int)

In [101]:
base_filtrado_lidos = pd.merge(recomendados, base1, on="Book Id")

In [ ]:
base_filtrado_lidos

In [103]:
base_resultado = base_filtrado_lidos[base_filtrado_lidos["LIDOS_y"] == "Não"]


In [104]:
print((base_resultado.loc[:, ['Book Id', 'TITULO_x', 'NOTA_MEDIA_x', 'QTD_TOTAL_AVALICAO_x']]).head(10))


    Book Id                                           TITULO_x  NOTA_MEDIA_x  \
0         8  Harry Potter Boxed Set  Books 1-5 (Harry Potte...          4.78   
3     15336                 The Lord of the Rings / The Hobbit          4.59   
4     17961                                 Collected Fictions          4.58   
6     15872  Harry Potter y el misterio del príncipe (Harry...          4.57   
8         5  Harry Potter and the Prisoner of Azkaban (Harr...          4.56   
9     43509  Harry Potter and the Goblet of Fire (Harry Pot...          4.56   
11    41908  Harry Potter und der Gefangene von Askaban (Ha...          4.56   
12    28087  Gabriel García Márquez: One Hundred Years of S...          4.54   
14    15876  Harry Potter y la Orden del Fénix (Harry Potte...          4.49   
15    15880                 Harrius Potter et Philosophi Lapis          4.47   

    QTD_TOTAL_AVALICAO_x  
0                  41428  
3                    141  
4                  18874  
6          

In [105]:
recomendados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 4 to 9966
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             486 non-null    int64  
 1   TITULO              486 non-null    object 
 2   Autor               486 non-null    object 
 3   NOTA_MEDIA          486 non-null    float64
 4   IDIOMA              486 non-null    object 
 5   NUM_TOTAL_PAG       486 non-null    int64  
 6   QTD_TOTAL_AVALICAO  486 non-null    int64  
 7   QTD_REVIEWS         486 non-null    int64  
 8   DATA_PUBLICACAO     486 non-null    object 
 9   EDITOR              486 non-null    object 
 10  GENEROS             486 non-null    object 
 11  LIDOS               486 non-null    object 
dtypes: float64(1), int64(4), object(7)
memory usage: 49.4+ KB


Apesar de não fazer muito "sentido" vou criar uma recomendação também pelo nome do livro lido por último, para isso vou utilizar a coluna criada aleatorio para pegar os últimos lidos. 

In [106]:
ultimo_lido = base1.loc[base1['LIDOS'] == 'Sim', 'TITULO'].iloc[-1]

In [112]:
#Importando as stopwords em inglês
nltk.download('stopwords')
#importação do modulo de adequação gramatical
nltk.download('punkt')
#importando e testando o modulo de wordnet que é para pegar os sinonimos das palavras
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
title_tokens = word_tokenize(ultimo_lido)
title_words = [lemmatizer.lemmatize(word.lower()) for word in title_tokens if word.lower() not in stop_words]

# Utilizar palavras-chave para recomendação baseada em conteúdo
corpus = recomendados['TITULO']
vectorizer = TfidfVectorizer()
#vetorizando as palavras
X = vectorizer.fit_transform(corpus)
title_vector = vectorizer.transform([' '.join(title_words)])
#calculando similaridade entre cosseno e vetor da palavra lida ( titulo )
cosine_similarities = cosine_similarity(title_vector, X).flatten()
#utilizando o asgort para trazer os 10 primeiros em relação 
related_docs_indices = cosine_similarities.argsort()[:-11:-1]
#trazendo os resultado
result = recomendados.iloc[related_docs_indices]

# ordenando lista de livros recomendados ordenados por relevância
recomendados = result.sort_values(by=['NOTA_MEDIA', 'QTD_TOTAL_AVALICAO'], ascending=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [113]:
recomendados

,Book Id,TITULO,Autor,NOTA_MEDIA,IDIOMA,NUM_TOTAL_PAG,QTD_TOTAL_AVALICAO,QTD_REVIEWS,DATA_PUBLICACAO,EDITOR,GENEROS,LIDOS
3899,13925,Child of the Prophecy (Sevenwaters #3),Juliet Marillier,4.12,eng,596,17071,708,6/16/2003,Tor Books,Fantasy|Romance|Historical|Historical Fiction|...,Não
3817,13658,The Other Wind (Earthsea Cycle #6),Ursula K. Le Guin/Samuel Roukin,4.12,eng,211,14393,635,9/30/2003,Ace Books,Fantasy|Fiction|Young Adult|Fantasy|Dragons|Fa...,Não
7486,28733,Take a Thief (Valdemar #25),Mercedes Lackey,4.12,eng,435,11911,174,10/1/2002,DAW,Fantasy|Fiction|Science Fiction Fantasy|Fantas...,Não
7227,27719,The Night of Wishes,Michael Ende/Heike Schwarzbauer/Rick Takvorian...,4.12,eng,216,3014,56,4/17/1995,Farrar Straus and Giroux (BYR),Fantasy|Childrens|Fiction|European Literature|...,Não
5237,18902,Book of Nightmares (Diadem Worlds of Magic #6),John Peel,4.12,eng,192,334,2,3/8/2005,Llewellyn Publications,Fantasy|Childrens|Middle Grade|Young Adult|You...,Não
3244,11901,The Witching Hour (Lives of the Mayfair Witche...,Anne Rice,4.11,eng,1207,87816,2577,11/4/2004,Arrow,Horror|Fantasy|Fiction|Fantasy|Paranormal|Para...,Não
3759,13570,Tsubasa: RESERVoir CHRoNiCLE Vol. 1,CLAMP/Anthony Gerard,4.11,eng,197,38296,396,4/27/2004,Del Rey,Sequential Art|Manga|Fantasy|Sequential Art|Gr...,Não
7409,28484,Into the Labyrinth (The Death Gate Cycle #6),Margaret Weis/Tracy Hickman,4.11,eng,441,18543,124,7/1/1994,Spectra Books,Fantasy|Fiction|Fantasy|Epic Fantasy|Science F...,Não
4672,16793,Stardust,Neil Gaiman,4.08,eng,248,283274,11746,8/29/2006,Harper Perennial,Fantasy|Fiction|Romance|Young Adult|Adventure|...,Não
9966,40152,Only Revolutions,Mark Z. Danielewski,3.21,eng,360,4791,443,9/12/2006,Pantheon Books,Fiction|Poetry|Fantasy|Romance|Novels|Literatu...,Não


In [114]:
recomendados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 3899 to 9966
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Book Id             10 non-null     int64  
 1   TITULO              10 non-null     object 
 2   Autor               10 non-null     object 
 3   NOTA_MEDIA          10 non-null     float64
 4   IDIOMA              10 non-null     object 
 5   NUM_TOTAL_PAG       10 non-null     int64  
 6   QTD_TOTAL_AVALICAO  10 non-null     int64  
 7   QTD_REVIEWS         10 non-null     int64  
 8   DATA_PUBLICACAO     10 non-null     object 
 9   EDITOR              10 non-null     object 
 10  GENEROS             10 non-null     object 
 11  LIDOS               10 non-null     object 
dtypes: float64(1), int64(4), object(7)
memory usage: 1.0+ KB
